In [8]:
import os
import nest_asyncio
import pandas as pd
from dotenv import find_dotenv, load_dotenv
from langsmith import Client
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.smith import RunEvalConfig, run_on_dataset

# To Avoid the Error on Jupyter Notebook (RuntimeError: This Event Loop Is Already Running)
# Patch Asyncio To Allow Nested Event Loops

nest_asyncio.apply()

In [7]:
# load_dotenv(find_dotenv())
os.environ["LANGCHAIN_API_KEY"] = str(os.getenv("Langchain_API"))
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "ATS"

client = Client()

# llm = ChatOpenAI()
llm.predict("Hello, world!")

c:\Users\Haide\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'Hello! How can I assist you today?'

Failed to batch ingest runs: LangSmithError('Failed to post https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')
Failed to batch ingest runs: LangSmithError('Failed to post https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')


In [9]:
example_inputs = [
    "a rap battle between Atticus Finch and Cicero",
    "a rap battle between Barbie and Oppenheimer",
    "a Pythonic rap battle between two swallows: one European and one African",
    "a rap battle between Aubrey Plaza and Stephen Colbert",
]

dataset_name = "Rap Battle Dataset"

# Storing inputs in a dataset lets us
# run chains and LLMs over a shared set of examples.
dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Rap battle prompts.",
)

for input_prompt in example_inputs:
    # Each example must be unique and have inputs defined.
    # Outputs are optional
    client.create_example(
        inputs={"question": input_prompt},
        outputs=None,
        dataset_id=dataset.id,
    )

HTTPError: [Errno 403 Client Error: Forbidden for url: https://api.smith.langchain.com/datasets] {"detail":"Forbidden"}

In [16]:
import langsmith
from langchain import chat_models, smith
from langchain import prompts
from langchain_core.output_parsers.string import StrOutputParser

# Define your runnable or chain below.
prompt = prompts.ChatPromptTemplate.from_messages(
  [
    ("system", "You are a helpful AI assistant."),
    ("human", "{your_input_key}")
  ]
)
llm = chat_models.ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
chain = prompt | llm | StrOutputParser()

# Define the evaluators to apply
eval_config = smith.RunEvalConfig(
    evaluators=[
        smith.RunEvalConfig.Criteria("helpfulness"),
        smith.RunEvalConfig.Criteria("controversiality")
    ],
    custom_evaluators=[],
    eval_llm=chat_models.ChatOpenAI(model="gpt-4", temperature=0)
)

client = langsmith.Client()
chain_results = client.run_on_dataset(
    dataset_name="System Prompts",
    llm_or_chain_factory=chain,
    evaluation=eval_config,
    project_name="ATS resume and job matching",
    concurrency_level=5,
    verbose=True,
)

LangSmithError: Failed to get https://api.smith.langchain.com/datasets in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/datasets?limit=1&name=System+Prompts', '{"detail":"Forbidden"}')

In [20]:
import openai
from langsmith.run_trees import RunTree
# This can be a user input to your app
question = "Can you summarize this morning's meetings?"
# Create a top-level run
pipeline = RunTree(
    name="Chat Pipeline",
    run_type="chain",
    inputs={"question": question}
)
# This can be retrieved in a retrieval step
context = "During this morning's meeting, we solved all world conflict."
messages = [
    { "role": "system", "content": "You are a helpful assistant. Please respond to the user's request only based on the given context." },
    { "role": "user", "content": f"Question: {question}\nContext: {context}"}
]
# Create a child run
child_llm_run = pipeline.create_child(
    name="OpenAI Call",
    run_type="llm",
    inputs={"messages": messages},
)
# Generate a completion
client = openai.Client()
chat_completion = client.chat.completions.create(
    model="gpt-3.5-turbo", messages=messages
)
# End the runs and log them
child_llm_run.end(outputs=chat_completion)
child_llm_run.post()
pipeline.end(outputs={"answer": chat_completion.choices[0].message.content})
pipeline.post()

Failed to batch ingest runs: LangSmithError('Failed to post https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')


In [4]:
from langsmith import Client  
from langsmith.evaluation import LangChainStringEvaluator, evaluate
import os

criteria_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "creativity": "Is this submission creative, imaginative, or novel?",
            "conciseness": "Is this response concise and to the point?",
        }
    }
)
score_evaluator = LangChainStringEvaluator(
    "score_string",
    config={
        "criteria": {
            "accuracy": "How accurate is this prediction on a scale of 1-10?" 
        },
        # If you want the score to be saved on a scale from 0 to 1
        "normalize_by": 10,
    }  
)

client = Client(api_key=os.getenv("LANGCHAIN_API"))
evaluate(
    data="<dataset_name>",
    evaluators=[
        criteria_evaluator,
        score_evaluator
    ], 
    metadata={"revision_id": "the version of your pipeline you are testing"},
)

This chain was only tested with GPT-4. Performance may be significantly worse with other models.


TypeError: evaluate() missing 1 required positional argument: 'target'

In [15]:
from llmlingua import PromptCompressor

prompt = """ You are an ATS (Applicant Tracking System) scanner which is designed to scan resume {0} for specific keywords and phrases. 
        Your task is to evaluate the resume against the provided job description {1}. Rate the resume between 1 - 10 where 1 being the lowest and 10 being the highest match with the job description. 
        The output should come as the rating / highest score, explain the reason for the score, missing keywords, lastly final thoughts with the list of best resources with links  to gain missing skills better if from the hiring company.
        """
llm_lingua = PromptCompressor("openai-community/gpt2", device_map="cpu")
compressed_prompt = llm_lingua.compress_prompt(prompt, instruction="", question="", target_token=200)

# > {'compressed_prompt': 'Question: Sam bought a dozen boxes, each with 30 highlighter pens inside, for $10 each box. He reanged five of boxes into packages of sixlters each and sold them $3 per. He sold the rest theters separately at the of three pens $2. How much did make in total, dollars?\nLets think step step\nSam bought 1 boxes x00 oflters.\nHe bought 12 * 300ters in total\nSam then took 5 boxes 6ters0ters.\nHe sold these boxes for 5 *5\nAfterelling these  boxes there were 3030 highlighters remaining.\nThese form 330 / 3 = 110 groups of three pens.\nHe sold each of these groups for $2 each, so made 110 * 2 = $220 from them.\nIn total, then, he earned $220 + $15 = $235.\nSince his original cost was $120, he earned $235 - $120 = $115 in profit.\nThe answer is 115',
#  'origin_tokens': 2365,
#  'compressed_tokens': 211,
#  'ratio': '11.2x',
#  'saving': ', Saving $0.1 in GPT-4.'}

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [10]:
compressed_prompt['compressed_prompt'] == prompt

True